## Crowdsourced segmentations

This is a notebook for exploring the [Large-scale crowdsourced radiotherapy segmentations across a variety of cancer sites](https://figshare.com/articles/dataset/Large-scale_crowdsourced_radiotherapy_segmentations_across_a_variety_of_cancer_sites/21074182).

In [ ]:
from pathlib import Path
from urllib.request import urlopen
from zipfile import ZipFile

from skrt import Patient, StructureSet

top_dir = Path("~/data/crowdsourced_segmentations").expanduser()
data_dir = top_dir / "Organized_files"

if not data_dir.exists():
    top_dir.mkdir(parents=True, exist_ok=True)
    data_url = "https://figshare.com/ndownloader/files/37841118"
    response = urlopen(data_url)
    data = response.read()
    zip_path = top_dir / "Organized_files.zip"
    with open(zip_path, "wb") as out_file:
        out_file.write(data)
    with ZipFile(zip_path) as zip_file:
        zip_file.extractall(top_dir)

In [ ]:
paths = [path for path in sorted(list(data_dir.iterdir())) if path.is_dir()]

patients = {}
for path in paths:
    if "Breast" not in str(path):
        continue
    patient = Patient(path, unsorted_dicom=True)
    patient.id = path.name
    structure_sets = patient.combined_objs("structure_set_types")
    experts = {}
    non_experts = {}
    for structure_set in structure_sets:
        structure_set.name = Path(structure_set.path).parent.parent.name
        if "Non-Expert" in structure_set.path:
            non_experts[structure_set.name] = structure_set
        else:
            experts[structure_set.name] = structure_set
    print(f"Read data for {patient.id} - experts: {len(experts)}; "
          f"non-experts: {len(non_experts)}; total: {len(structure_sets)}")
    images = patient.combined_objs("image_types")
    assert len(images) == 1
    assert images[0].get_dicom_dataset().Modality == "CT"
    patient.ct_image = images[0]
    patient.experts = {key: value for key, value in sorted(experts.items())}
    patient.non_experts = {key: value for key, value in sorted(non_experts.items())}
    patients[patient.id] = patient

In [ ]:
from skrt import StructureSet
patient = patients["Breast"]
print(patient.experts.keys())

In [ ]:
#rois = StructureSet()
#for expert, ss in list(patient.experts.items()):
#    rois += StructureSet(ss["CTV_IMN"], name=ss.name)
#rois.set_image(patient.ct_image)
roi = "CTV_IMN"
rois = [StructureSet(ss[roi]) for ss in patient.experts.values()]
imns = sum(rois, StructureSet())
imns.set_image(patient.ct_image)

In [ ]:
print(imns.get_roi_names())

In [ ]:
print(imns.name)

In [ ]:
print(imns.summed_names)

In [ ]:
patient.ct_image.view(rois=imns)